# Predict Workout

Predict Workout Day and distance

In [ ]:
import pandas as pd

filepath = "data/"

def loadCleanData():
    df = pd.read_csv(filepath+ 'activities_clean.csv')
    df['Week'] = pd.to_datetime(df.Week)
    return df

data = loadCleanData()
data.head()